# Problem Statement
## Introduction

Apply EDA techniques to analyse and make conclusions about loan data of a consumer finance company.

* Understading of risk analytics in banking and financial service and how the data can be used to minimize the risk of losing money while lending


  

## Business Understanding
The company has to make a decision for loan approval based on the applicant’s profile. Two types of risks are associated with the bank’s decision:

* If the applicant is likely to repay the loan, then not approving the loan results in a loss of business to the company

* If the applicant is not likely to repay the loan, i.e. he/she is likely to default, then approving the loan may lead to a financial loss for the company

 
The data given below contains information about past loan applicants and whether they ‘defaulted’ or not. The aim is to identify patterns which indicate if a person is likely to default, which may be used for taking actions such as **denying the loan, reducing the amount of loan, lending (to risky applicants) at a higher interest rate**, etc.

 

Use EDA to understand how consumer attributes and loan attributes influence the tendency of default.

When a person applies for a loan, there are two types of decisions that could be taken by the company:

### Loan accepted: If the company approves the loan, there are 3 possible scenarios described below:

#### Fully paid: Applicant has fully paid the loan (the principal and the interest rate)

##### Current: Applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed. These candidates are not labelled as 'defaulted'.

##### Charged-off: Applicant has not paid the instalments in due time for a long period of time, i.e. he/she has defaulted on the loan 

### Loan rejected: The company had rejected the loan (because the candidate does not meet their requirements etc.). Since the loan was rejected, there is no transactional history of those applicants with the company and so this data is not available with the company (and thus in this dataset)
 

## Business Objectives
This company is the largest online loan marketplace, facilitating personal loans, business loans, and financing of medical procedures. Borrowers can easily access lower interest rate loans through a fast online interface. 

 

Like most other lending companies, lending loans to ‘risky’ applicants is the largest source of financial loss (called credit loss). Credit loss is the amount of money lost by the lender when the borrower refuses to pay or runs away with the money owed. In other words, borrowers who default cause the largest amount of loss to the lenders. In this case, the **customers labelled as 'charged-off' are the 'defaulters'. **

 

If one is able to identify these risky loan applicants, then such loans can be reduced thereby cutting down the amount of credit loss. Identification of such applicants using EDA is the aim of this case study.

 

In other words, the company wants to understand **the driving factors (or driver variables) behind loan default, i.e. the variables which are strong indicators of default.**  The company can utilise this knowledge for its portfolio and risk assessment. 


** research a little about risk analytics** (understanding the types of variables and their significance should be enough).

 

## Data Understanding
 



In [1]:
import pandas as pd

filename = "loan.csv"

loan_data = pd.read_csv(filename)

C:\Users\Bineesh\AppData\Local\Temp\ipykernel_20876\1607258213.py:5: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  loan_data = pd.read_csv(filename)


In [2]:
loan_data.columns

Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade',
       ...
       'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq',
       'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens',
       'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit',
       'total_il_high_credit_limit'],
      dtype='object', length=111)

* Load data contains 111 columns

In [3]:
filename1 = "Data_Dictionary.xlsx"
data_dictionary = pd.read_excel(filename1)




In [4]:
data_dictionary.columns
loanstat = 'loanstat'
description = 'description'
data_dictionary.columns = [loanstat, description]
data_dictionary

,loanstat,description
0,acc_now_delinq,The number of accounts on which the borrower i...
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan...
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by th...
...,...,...
112,verification_status,"Indicates if income was verified by LC, not ve..."
113,verified_status_joint,Indicates if the co-borrowers' joint income wa...
114,zip_code,The first 3 numbers of the zip code provided b...
115,NaN,NaN


* data dictionary contains 117 rows which corresponds to 111 columns in the load dataframe
* So need to correct this data quality issue

In [5]:
column_df = pd.DataFrame({loanstat: loan_data.columns.sort_values()})
column_df

,loanstat
0,acc_now_delinq
1,acc_open_past_24mths
2,addr_state
3,all_util
4,annual_inc
...,...
106,total_rev_hi_lim
107,url
108,verification_status
109,verification_status_joint


In [6]:
def get_column_mismatch(loan_data, data_dictionary):
    diff1 = set(loan_data.columns) - set(data_dictionary[loanstat])
    print("These are the extra columns in loan_data\n", diff1)
    diff2 = set(data_dictionary[loanstat]) - set(loan_data.columns)
    print("These are the extra info in data dictionary\n", diff2)
    #return diff1, diff2
    
get_column_mismatch(loan_data, data_dictionary)


These are the extra columns in loan_data
 {'total_rev_hi_lim', 'verification_status_joint'}
These are the extra info in data dictionary
 {'fico_range_high', nan, 'fico_range_low', 'last_fico_range_high', 'total_rev_hi_lim \xa0', 'last_fico_range_low', 'verified_status_joint'}


* There are two columns extra in the loan data.

* There are two columns extra in the loan data.
* There is an issue with total_rev_hi_lim, ie some special characters in it
* verification_status_joint in loan_data is verified_status_joint in data dictionary
* There are nan values in the data dictionary


In [7]:
mismatched_values = {'total_rev_hi_lim \xa0': 'total_rev_hi_lim', 'verified_status_joint': 'verification_status_joint'}
data_dictionary[loanstat] = data_dictionary[loanstat].replace(mismatched_values)

get_column_mismatch(loan_data, data_dictionary)

These are the extra columns in loan_data
 set()
These are the extra info in data dictionary
 {'fico_range_high', nan, 'fico_range_low', 'last_fico_range_high', 'last_fico_range_low'}


* 'fico_range_high',
  'fico_range_low',
  'last_fico_range_high',
  'last_fico_range_low' These 4 variables to be interpreted later. These are present in data dictionary but not present in loan data. Need to check whether these variables can be derived from existing variables

In [30]:
def create_data_dictionary(loan_data, data_dictionary, filename="cleaned_data_dictionary.csv"):

    column_df = pd.DataFrame({loanstat: loan_data.columns.sort_values()})
    cleaned_data_dictionary = column_df.merge(data_dictionary, on=loanstat, how='inner')
    cleaned_data_dictionary.to_csv(filename, index=False)
    return cleaned_data_dictionary.set_index(loanstat)[description].to_dict()
    
def get_general_info(loan_data):
    display(loan_data.shape)
    display(loan_data.info())
    display(loan_data.describe())
    

In [17]:
create_data_dictionary(loan_data, data_dictionary)


{'acc_now_delinq': 'The number of accounts on which the borrower is now delinquent.',
 'acc_open_past_24mths': 'Number of trades opened in past 24 months.',
 'addr_state': 'The state provided by the borrower in the loan application',
 'all_util': 'Balance to credit limit on all trades',
 'annual_inc': 'The self-reported annual income provided by the borrower during registration.',
 'annual_inc_joint': 'The combined self-reported annual income provided by the co-borrowers during registration',
 'application_type': 'Indicates whether the loan is an individual application or a joint application with two co-borrowers',
 'avg_cur_bal': 'Average current balance of all accounts',
 'bc_open_to_buy': 'Total open to buy on revolving bankcards.',
 'bc_util': 'Ratio of total current balance to high credit/credit limit for all bankcard accounts.',
 'chargeoff_within_12_mths': 'Number of charge-offs within 12 months',
 'collection_recovery_fee': 'post charge off collection fee',
 'collections_12_mth

In [10]:
loan_data.shape

(39717, 111)

In [11]:
(loan_data.isnull().sum() == loan_data.shape[0]).sum()

54

* 54 columns are completely empty


In [12]:
# get rid of 54 empty columns

columns_non_null = (loan_data.isnull().sum() != loan_data.shape[0])
columns_non_null

loan_data1 = loan_data.loc[:,columns_non_null]

In [13]:
loan_data1.shape

(39717, 57)

In [14]:
for i in range(0, len(data_dictionary)):
    variable = data_dictionary.iloc[i,0]
    if variable in loan_data1.columns:
        shape = loan_data1[variable].value_counts().shape
        print(variable, shape)
#         if shape[0] == 1:
#             print(variable, shape, loan_data1[variable].value_counts())

acc_now_delinq (1,)
addr_state (50,)
annual_inc (5318,)
application_type (1,)
chargeoff_within_12_mths (1,)
collection_recovery_fee (2616,)
collections_12_mths_ex_med (1,)
delinq_2yrs (11,)
delinq_amnt (1,)
desc (26527,)
dti (2868,)
earliest_cr_line (526,)
emp_length (11,)
emp_title (28820,)
funded_amnt (1041,)
funded_amnt_inv (8205,)
grade (7,)
home_ownership (5,)
id (39717,)
initial_list_status (1,)
inq_last_6mths (9,)
installment (15383,)
int_rate (371,)
issue_d (55,)
last_credit_pull_d (106,)
last_pymnt_amnt (34930,)
last_pymnt_d (101,)
loan_amnt (885,)
loan_status (3,)
member_id (39717,)
mths_since_last_delinq (95,)
mths_since_last_record (111,)
next_pymnt_d (2,)
open_acc (40,)
out_prncp (1137,)
out_prncp_inv (1138,)
policy_code (1,)
pub_rec (5,)
pub_rec_bankruptcies (3,)
purpose (14,)
pymnt_plan (1,)
recoveries (4040,)
revol_bal (21711,)
revol_util (1089,)
sub_grade (35,)
tax_liens (1,)
term (2,)
title (19615,)
total_acc (82,)
total_pymnt (37850,)
total_pymnt_inv (37518,)
total_r

In [19]:
columns_with_single_unique_value = loan_data1.columns[loan_data1.nunique() == 1]
columns_with_single_unique_value

Index(['pymnt_plan', 'initial_list_status', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens'],
      dtype='object')

In [20]:
loan_data1[columns_with_single_unique_value].isnull().sum()

pymnt_plan                     0
initial_list_status            0
collections_12_mths_ex_med    56
policy_code                    0
application_type               0
acc_now_delinq                 0
chargeoff_within_12_mths      56
delinq_amnt                    0
tax_liens                     39
dtype: int64

In [21]:
## identify the columns where there are only single unique value but also have null values

# Even though the non presence of a data(or null value might be an indicator)

columns_with_single_values_detail = loan_data1[columns_with_single_unique_value].isnull().sum() == 0
columns_with_single_values_detail

pymnt_plan                     True
initial_list_status            True
collections_12_mths_ex_med    False
policy_code                    True
application_type               True
acc_now_delinq                 True
chargeoff_within_12_mths      False
delinq_amnt                    True
tax_liens                     False
dtype: bool

* Three columns collections_12_mths_ex_med, chargeoff_within_12_mths, tax_liens
* For the time being delete these columns as well

In [23]:
loan_data2 = loan_data1.drop(columns=columns_with_single_values_detail.index)

In [24]:
loan_data2.columns


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'pub_rec_bankruptcies'],
      dtype='object')

In [25]:
loan_data2['loan_status'].value_counts()

Fully Paid     32950
Charged Off     5627
Current         1140
Name: loan_status, dtype: int64

In [31]:
get_general_info(loan_data2)

(39717, 48)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 48 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       39717 non-null  int64  
 1   member_id                39717 non-null  int64  
 2   loan_amnt                39717 non-null  int64  
 3   funded_amnt              39717 non-null  int64  
 4   funded_amnt_inv          39717 non-null  float64
 5   term                     39717 non-null  object 
 6   int_rate                 39717 non-null  object 
 7   installment              39717 non-null  float64
 8   grade                    39717 non-null  object 
 9   sub_grade                39717 non-null  object 
 10  emp_title                37258 non-null  object 
 11  emp_length               38642 non-null  object 
 12  home_ownership           39717 non-null  object 
 13  annual_inc               39717 non-null  float64
 14  verification_status   

None

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
count,3.971700e+04,3.971700e+04,39717.000000,39717.000000,39717.000000,39717.000000,3.971700e+04,39717.000000,39717.000000,39717.000000,...,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39020.000000
mean,6.831319e+05,8.504636e+05,11219.443815,10947.713196,10397.448868,324.561922,6.896893e+04,13.315130,0.146512,0.869200,...,50.989768,12153.596544,11567.149118,9793.348813,2263.663172,1.363015,95.221624,12.406112,2678.826162,0.043260
std,2.106941e+05,2.656783e+05,7456.670694,7187.238670,7128.450439,208.874874,6.379377e+04,6.678594,0.491812,1.070219,...,373.824457,9042.040766,8942.672613,7065.522127,2608.111964,7.289979,688.744771,148.671593,4447.136012,0.204324
min,5.473400e+04,7.069900e+04,500.000000,500.000000,0.000000,15.690000,4.000000e+03,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.162210e+05,6.667800e+05,5500.000000,5400.000000,5000.000000,167.020000,4.040400e+04,8.170000,0.000000,0.000000,...,0.000000,5576.930000,5112.310000,4600.000000,662.180000,0.000000,0.000000,0.000000,218.680000,0.000000
50%,6.656650e+05,8.508120e+05,10000.000000,9600.000000,8975.000000,280.220000,5.900000e+04,13.400000,0.000000,1.000000,...,0.000000,9899.640319,9287.150000,8000.000000,1348.910000,0.000000,0.000000,0.000000,546.140000,0.000000
75%,8.377550e+05,1.047339e+06,15000.000000,15000.000000,14400.000000,430.780000,8.230000e+04,18.600000,0.000000,1.000000,...,0.000000,16534.433040,15798.810000,13653.260000,2833.400000,0.000000,0.000000,0.000000,3293.160000,0.000000
max,1.077501e+06,1.314167e+06,35000.000000,35000.000000,35000.000000,1305.190000,6.000000e+06,29.990000,11.000000,8.000000,...,6307.370000,58563.679930,58563.680000,35000.020000,23563.680000,180.200000,29623.350000,7002.190000,36115.200000,2.000000


In [34]:
cleaned_data_dictionary = create_data_dictionary(loan_data2, data_dictionary)
cleaned_data_dictionary

{'addr_state': 'The state provided by the borrower in the loan application',
 'annual_inc': 'The self-reported annual income provided by the borrower during registration.',
 'collection_recovery_fee': 'post charge off collection fee',
 'delinq_2yrs': "The number of 30+ days past-due incidences of delinquency in the borrower's credit file for the past 2 years",
 'desc': 'Loan description provided by the borrower',
 'dti': 'A ratio calculated using the borrower’s total monthly debt payments on the total debt obligations, excluding mortgage and the requested LC loan, divided by the borrower’s self-reported monthly income.',
 'earliest_cr_line': "The month the borrower's earliest reported credit line was opened",
 'emp_length': 'Employment length in years. Possible values are between 0 and 10 where 0 means less than one year and 10 means ten or more years. ',
 'emp_title': 'The job title supplied by the Borrower when applying for the loan.*',
 'funded_amnt': 'The total amount committed to 

In [71]:
def equal(loan_data, column1, column2):
    equality = loan_data[column1].equals(loan_data[column2])
    print(column1, column2, equality)
    return equality
    
def analysis(loan_data, cleaned_data_dictionary):
    loan_data_modified = loan_data.copy()
    columns_to_drop = ['id', 'member_id']
    description = [cleaned_data_dictionary[column] for column in columns_to_drop]
    reason_to_drop = ['irrelevant', 'irrelevant']
    columns_to_keep = ['funded_amnt', 'funded_amnt_inv']
    columns_to_modify = [('term', 'convert to int after replacing months with empty string. we have only 36 months and 60 months')]
#     for column in loan_data.columns:
#         str1 = f"{column}: {cleaned_data_dictionary[column]}" 
#         print(str1)
#     print(columns_to_drop)
#     index = 3
#     column = loan_data.columns[index]
#     str1 = f"{column}: {cleaned_data_dictionary[column]}"
#     print(str1)

    equal(loan_data, 'funded_amnt', 'funded_amnt_inv')
    print('funded_amnt', 'funded_amnt_inv', 'seem to have different values')
    
    
    display(loan_data.columns)
    column = 'term'
    print(loan_data[column].value_counts())
    loan_data_modified[column] = loan_data[column].str.replace(' months', '').astype(int)
    
    loan_data_modified = loan_data_modified.drop(columns=columns_to_drop)
    print(loan_data_modified[column])
    get_general_info(loan_data_modified)
analysis(loan_data2, cleaned_data_dictionary)

funded_amnt funded_amnt_inv False
funded_amnt funded_amnt_inv seem to have different values


Index(['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv',
       'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title',
       'emp_length', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'url', 'desc', 'purpose', 'title', 'zip_code',
       'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt',
       'next_pymnt_d', 'last_credit_pull_d', 'pub_rec_bankruptcies'],
      dtype='object')

 36 months    29096
 60 months    10621
Name: term, dtype: int64
0        36
1        60
2        36
3        36
4        60
         ..
39712    36
39713    36
39714    36
39715    36
39716    36
Name: term, Length: 39717, dtype: int32


(39717, 46)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39717 entries, 0 to 39716
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   loan_amnt                39717 non-null  int64  
 1   funded_amnt              39717 non-null  int64  
 2   funded_amnt_inv          39717 non-null  float64
 3   term                     39717 non-null  int32  
 4   int_rate                 39717 non-null  object 
 5   installment              39717 non-null  float64
 6   grade                    39717 non-null  object 
 7   sub_grade                39717 non-null  object 
 8   emp_title                37258 non-null  object 
 9   emp_length               38642 non-null  object 
 10  home_ownership           39717 non-null  object 
 11  annual_inc               39717 non-null  float64
 12  verification_status      39717 non-null  object 
 13  issue_d                  39717 non-null  object 
 14  loan_status           

None

,loan_amnt,funded_amnt,funded_amnt_inv,term,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,mths_since_last_delinq,...,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,pub_rec_bankruptcies
count,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,3.971700e+04,39717.000000,39717.000000,39717.000000,14035.000000,...,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39717.000000,39020.000000
mean,11219.443815,10947.713196,10397.448868,42.418007,324.561922,6.896893e+04,13.315130,0.146512,0.869200,35.900962,...,50.989768,12153.596544,11567.149118,9793.348813,2263.663172,1.363015,95.221624,12.406112,2678.826162,0.043260
std,7456.670694,7187.238670,7128.450439,10.622815,208.874874,6.379377e+04,6.678594,0.491812,1.070219,22.020060,...,373.824457,9042.040766,8942.672613,7065.522127,2608.111964,7.289979,688.744771,148.671593,4447.136012,0.204324
min,500.000000,500.000000,0.000000,36.000000,15.690000,4.000000e+03,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5500.000000,5400.000000,5000.000000,36.000000,167.020000,4.040400e+04,8.170000,0.000000,0.000000,18.000000,...,0.000000,5576.930000,5112.310000,4600.000000,662.180000,0.000000,0.000000,0.000000,218.680000,0.000000
50%,10000.000000,9600.000000,8975.000000,36.000000,280.220000,5.900000e+04,13.400000,0.000000,1.000000,34.000000,...,0.000000,9899.640319,9287.150000,8000.000000,1348.910000,0.000000,0.000000,0.000000,546.140000,0.000000
75%,15000.000000,15000.000000,14400.000000,60.000000,430.780000,8.230000e+04,18.600000,0.000000,1.000000,52.000000,...,0.000000,16534.433040,15798.810000,13653.260000,2833.400000,0.000000,0.000000,0.000000,3293.160000,0.000000
max,35000.000000,35000.000000,35000.000000,60.000000,1305.190000,6.000000e+06,29.990000,11.000000,8.000000,120.000000,...,6307.370000,58563.679930,58563.680000,35000.020000,23563.680000,180.200000,29623.350000,7002.190000,36115.200000,2.000000


In [69]:
column = 'term'
loan_data2[column].str.replace(' months', '').astype(int)

0        36
1        60
2        36
3        36
4        60
         ..
39712    36
39713    36
39714    36
39715    36
39716    36
Name: term, Length: 39717, dtype: int32